In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import pandas as pd
import time
from itertools import chain
import random

import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

In [2]:
Userdf = pd.DataFrame.from_csv('UserReviews.csv')

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [3]:
Gamesdf = pd.DataFrame.from_csv('AllGames.csv')

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [4]:
Top30 = Userdf.Game_Title.value_counts().index[0:30]

In [5]:
UserID = []
Game_Title = []
Rating = []

In [7]:
print('Please enter a username \n')
username = input()

print('\n')
GamestobeRanked = random.sample(list(Top30), k=4)
print('Consider the following games. \nPlease rank your interest in these games from 1 to 4. 1 is the best, and 4 is the worst. \n')
print('\n')
print(GamestobeRanked[0]+'\n')
print(GamestobeRanked[1]+'\n')
print(GamestobeRanked[2]+'\n')
print(GamestobeRanked[3]+'\n')
print('\n')
print(GamestobeRanked[0] + ' is ranked number:')
Ranking0 = int(input())
print(GamestobeRanked[1] + ' is ranked number:')
Ranking1 = int(input())
print(GamestobeRanked[2]+ ' is ranked number:')
Ranking2 = int(input())
print(GamestobeRanked[3]+ ' is ranked number:')
Ranking3 =  int(input())

Please enter a username 

SamBell


Consider the following games. 
Please rank your interest in these games from 1 to 4. 1 is the best, and 4 is the worst. 



Pokemon Sword

The Witcher 3: Wild Hunt - Complete Edition

Splatoon 2

Mario + Rabbids: Kingdom Battle



Pokemon Sword is ranked number:
2
The Witcher 3: Wild Hunt - Complete Edition is ranked number:
1
Splatoon 2 is ranked number:
3
Mario + Rabbids: Kingdom Battle is ranked number:
4


In [8]:
Games = GamestobeRanked[0:4]
Rankings = [Ranking0, Ranking1, Ranking2, Ranking3]

In [9]:
def _replaceitem(x):
    if x == 1:
        return 7
    elif x == 2:
        return 6
    elif x == 3:
        return 5
    else:
        return x

Rankings = map(_replaceitem, Rankings)

In [10]:
for Game in Games:
    UserID.append(username)
Game_Title.extend(Games)
Rating.extend(Rankings)

In [11]:
UserRatings = pd.DataFrame([UserID, Game_Title, Rating]).transpose()
UserRatings.columns = ['UserID', 'Game_Title', 'Rating']

In [12]:
UserRatings

,UserID,Game_Title,Rating
0,SamBell,Pokemon Sword,6
1,SamBell,The Witcher 3: Wild Hunt - Complete Edition,7
2,SamBell,Splatoon 2,5
3,SamBell,Mario + Rabbids: Kingdom Battle,4


In [13]:
Userdf = Userdf.append(UserRatings, ignore_index=True)

In [14]:
# Instantiate a 'Reader' to read in the data so Surprise can use it
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(Userdf[['UserID', 'Game_Title', 'Rating']], reader)

In [15]:
trainset, testset = train_test_split(data, test_size=.2)

In [16]:
final = SVD(n_epochs=20, n_factors=1, biased=True, 
              lr_all=0.005, reg_all=0.06)

In [17]:
final.fit(trainset)

In [18]:
predictions = final.test(testset)

In [19]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 2.3976
MAE:  1.6505


1.6504928703941577

In [20]:
from collections import defaultdict

from surprise import SVD
# from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        

    return top_n


In [21]:
# First train an SVD algorithm on the dataset.
trainset = data.build_full_trainset()
algo = SVD()
model = algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

In [22]:
import os

from surprise import SVD
from surprise import dump
from surprise.accuracy import rmse

In [23]:
import pickle

In [24]:
pickle.dump( predictions, open( "predictions", "wb" ) )

In [26]:
infile = open('predictions','rb')
predictions = pickle.load(infile)


In [468]:
print(new_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
print('\n Consider the following games: \n' + GamestobeRanked + ' \nPlease rank your interest in these games from 1 to 4. 1 is the best, and 4 is the worst:\n')

TypeError: can only concatenate str (not "list") to str

In [462]:
predictions

[Prediction(uid='greedyflygon1', iid='Super Smash Bros. Ultimate', r_ui=7.8945538818076475, est=9.56267722158186, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid='Super Mario Odyssey', r_ui=7.8945538818076475, est=9.086430523851792, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid='Mario Kart 8 Deluxe', r_ui=7.8945538818076475, est=9.474341646262666, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid='Pokemon Shield', r_ui=7.8945538818076475, est=4.615578793292697, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid='The Legend of Zelda: Breath of the Wild', r_ui=7.8945538818076475, est=8.817382302621676, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid='The Elder Scrolls V: Skyrim', r_ui=7.8945538818076475, est=8.069442967308614, details={'was_impossible': False}),
 Prediction(uid='greedyflygon1', iid="Luigi's Mansion 3", r_ui=7.8945538818076475, est=8.602343745565738, d

In [27]:
print('According to your rankings, these are your recommended games:')
print(username)
print("1. " + top_n.get(username)[0][0])
print("2. " + top_n.get(username)[1][0])
print("3. " + top_n.get(username)[2][0])
print("4. " + top_n.get(username)[3][0])
print("5. " + top_n.get(username)[4][0])
print("6. " + top_n.get(username)[5][0])
print("7. " + top_n.get(username)[6][0])
print("8. " + top_n.get(username)[7][0])
print("9. " + top_n.get(username)[8][0])
print("10. " + top_n.get(username)[9][0])

According to your rankings, these are your recommended games:
SamBell
1. The Legend of Zelda: Breath of the Wild
2. Celeste
3. Enter the Gungeon
4. Mario Kart 8 Deluxe
5. Cuphead
6. Super Smash Bros. Ultimate
7. The Legend of Zelda: Link's Awakening
8. Astral Chain
9. Stardew Valley
10. Diablo III: Eternal Collection


In [ ]:
infile.close()